In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

make test input yaml

In [5]:
template = {
    'input_tif': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/sandbox/test_images/test/test.ome.tiff'
    },
    'spatial_features': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/sandbox/test_images/test/output_spatial_features.h5ad'
    },
#     'mask_tif': {
#         'class': 'File',
#         'path': ''
#     },
#     'mask_markers': ''
}

In [8]:
yaml.safe_dump(template, open('../tests/data/region_features/inputs_region_features.yaml', 'w'))

make cromwell compute1 config

In [7]:
# !mkdir ../tests/data/region_features

In [9]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/region_features'
output_fp = '../tests/data/region_features/region_features.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

make execution script

In [10]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/region_features'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'region_features.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'region_features.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'inputs_region_features.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab', '/scratch1/fs1/dinglab']


In [11]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [12]:
run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root, log_fp='log.txt')
filepath = os.path.join('../tests/data/region_features/run_job.sh')
bsub.write_command_file(run_commands, filepath)

In [13]:
for c in run_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/region_features/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/region_features:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/region_features /storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/region_features:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/region_features /storage1/fs1/dinglab:/storage1/fs1/dinglab /scratch1/fs1/dinglab:/scratch1/fs1/dinglab"
bgadd -L 10 /estorrs/test_cromwell
export PATH="/opt/java/openjdk/bin:$PATH"
bsub -R 'select[mem>10GB] rusage[mem=10GB] span[hosts=1]' -M 11GB -n 1 -q dinglab -G compute-dinglab -a 'docker(estorrs/cromwell-runner:58)' -g /estorrs/test_cromwell -J 7d0a